In [2]:
""" 유방암 데이터 정리 """
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

df = pd.read_csv('../dataset/wdbc.data', header = None)
X = df.loc[:, 2:].values
y = df.loc[:, 1].values

# 클래스 레이블 문자열을 정수로 변환
le = LabelEncoder()
y = le.fit_transform(y)
print(le.classes_)
print(le.transform(['M', 'B']))

# 데이터 쪼개기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, stratify = y, random_state = 1)

['B' 'M']
[1 0]


In [4]:
""" 그리드 서치를 사용해 세부 튜닝하기 """

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import numpy as np

pipe_svc = make_pipeline(StandardScaler(), SVC (random_state = 1))
param_range = [0.0001, 0.001, 0.01, 0.1,
              1.0, 10.0, 100.0, 1000.0]
param_grid = [{'svc__C' : param_range, 'svc__kernel' : ['linear']},
             {'svc__C' : param_range, 'svc__gamma' : param_range, 'svc__kernel' : ['rbf']}]
gs = GridSearchCV(estimator = pipe_svc, param_grid = param_grid, scoring = 'accuracy',
                 cv = 10, n_jobs = 1)
gs = gs.fit(X_train, y_train)
print(gs.best_score_)
print(gs.best_params_)

0.9846859903381642
{'svc__C': 100.0, 'svc__gamma': 0.001, 'svc__kernel': 'rbf'}


In [5]:
""" 테스트 세트로 최고 모델의 성능 추정 """

clf = gs.best_estimator_
clf.fit(X_train, y_train)
print('테스트 정확도 : %.3f' % clf.score(X_test, y_test))

테스트 정확도 : 0.974


In [7]:
""" 중첩 교차 검증 """
from sklearn.model_selection import cross_val_score

gs = GridSearchCV(estimator = pipe_svc, param_grid = param_grid,
                 scoring = 'accuracy', cv = 2)
scores = cross_val_score(gs, X_train, y_train, scoring = 'accuracy', cv = 5)
print('CV 정확도 : %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

CV 정확도 : 0.974 +/- 0.015


In [10]:
""" 결정트리에 적용해보기 """
from sklearn.tree import DecisionTreeClassifier

gs = GridSearchCV(estimator = DecisionTreeClassifier(random_state = 0),
                 param_grid = [{'max_depth' : [1, 2, 3, 4, 5, 6, 7, None]}],
                 scoring = 'accuracy', cv = 2)
scores = cross_val_score(gs, X_train, y_train, scoring = 'accuracy', cv = 5)
print('CV 정확도 : %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

print('따라서 이 데이터에는 SVM이 나은 판단임을 알 수 있다.')

CV 정확도 : 0.934 +/- 0.016
따라서 이 데이터에는 SVM이 나은 판단임을 알 수 있다.
